In [98]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader, Subset
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
import optuna
import os

from sklearn.model_selection import KFold

print(torch.__version__)

2.5.1


In [99]:
# Define relevant variables for the ML task
num_classes = 10
# Device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the Dataset and Transformations

In [100]:
# Train transformations for augmenting training data
train_transforms = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize all images to a fixed size of 32x32 pixels, as LeNet requires consistent input dimensions.
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally with a probability of 0.5 to increase dataset diversity.
    transforms.RandomRotation(10),  # Randomly rotate the image by up to ±10 degrees to simulate real-world variations in orientation.
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # Apply random affine transformations including shearing (up to ±10 degrees) and scaling (randomly resize within 80%-120%).
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor and normalize the pixel values to a range [0, 1].
    transforms.Normalize((0.5,), (0.5,)),  # Normalize the tensor values to have mean 0 and standard deviation 1 based on the specified mean and std (applied channel-wise).
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)  # Introduce random changes in brightness, contrast, saturation, and hue to create variability in lighting conditions.
])

# Test transformations to process test data consistently with the model's expectations
test_transforms = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize all test images to a fixed size of 32x32 pixels for compatibility with the model.
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor and scale pixel values to [0, 1].
    transforms.Normalize((0.5,), (0.5,))  # Normalize the test tensor values using mean 0 and std 1 to align with the training data input.
])

# Dataset initialization
dataset = torchvision.datasets.MNIST(
    root='./data',  # Path to the directory where the dataset should be downloaded or found.
    train=True,  # Indicates that we want to load the training set. If set to False, it would load the test set.
    transform=train_transforms,  # Apply the defined transformations to preprocess the training data.
    download=True  # Ensure the dataset is downloaded from the source if it's not already present in the specified directory.
    # This is useful to automatically fetch the MNIST dataset the first time the script is run.
)

# Define Model

In [101]:
class LeNet(nn.Module):
    """
    Implementation of the LeNet model for image classification.
    LeNet is a classic Convolutional Neural Network (CNN) architecture,
    which consists of convolutional layers for feature extraction,
    activation functions for introducing non-linearity, pooling layers for
    dimensionality reduction, and fully connected layers for the final classification.

    Key components:
    - Convolutional layers: Learn spatial hierarchies of features in the data.
    - BatchNormalization: Stabilizes and accelerates training.
    - Activation functions: Introduce non-linearity to the network.
    - MaxPooling: Reduces spatial dimensions and helps prevent overfitting.
    - Fully connected layers: Perform the classification task.
    """
    def __init__(self, num_classes, fc1_size, fc2_size):
        """
        Initializes the LeNet model with configurable fully connected layer sizes.

        Args:
        - num_classes (int): The number of output classes for classification.
        - fc1_size (int): The number of neurons in the first fully connected layer.
        - fc2_size (int): The number of neurons in the second fully connected layer.
        """
        super(LeNet, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(
            in_channels=1,  # Input has 1 channel (grayscale image)
            out_channels=6,  # Output has 6 feature maps
            kernel_size=5,  # Kernel size is 5x5
            stride=1,  # Stride of 1 (no skipping)
            padding=0  # No padding applied
        )
        self.relu1 = nn.ReLU()  # Activation function (ReLU) adds non-linearity
        self.bn1 = nn.BatchNorm2d(6)  # Batch normalization to stabilize and improve convergence

        # Second convolutional layer
        self.conv2 = nn.Conv2d(
            in_channels=6,  # Input has 6 feature maps from conv1
            out_channels=16,  # Output has 16 feature maps
            kernel_size=5,  # Kernel size is 5x5
            stride=1,  # Stride of 1
            padding=0  # No padding
        )
        self.relu2 = nn.ReLU()  # Activation function (ReLU) adds non-linearity
        self.bn2 = nn.BatchNorm2d(16)  # Batch normalization after second activation

        # Fully connected (dense) layers
        self.fc1 = nn.Linear(
            in_features=16 * 5 * 5,  # Input size after flattening (16 feature maps of size 5x5)
            out_features=fc1_size  # Number of neurons in first fully connected layer
        )
        self.relu3 = nn.ReLU()  # Activation function (ReLU) introduces non-linearity

        self.fc2 = nn.Linear(
            in_features=fc1_size,  # Input size is the output from the first FC layer
            out_features=fc2_size  # Number of neurons in second fully connected layer
        )
        self.relu4 = nn.ReLU()  # Activation function (ReLU) introduces non-linearity

        self.fc3 = nn.Linear(
            in_features=fc2_size,  # Input size is the output from the second FC layer
            out_features=num_classes  # Number of output classes for classification
        )
        # No activation (e.g., softmax) here because CrossEntropyLoss expects raw logits.

    def forward(self, x):
        """
        Defines the forward pass for the LeNet model.

        Args:
        - x (Tensor): Input tensor representing the batch of images.

        Returns:
        - x (Tensor): The raw logits corresponding to the classification outputs.
        """
        # Apply the first convolutional layer, activation, batch normalization, and max pooling
        x = self.conv1(x)  # Convolution operation
        x = self.relu1(x)  # Apply ReLU activation
        x = self.bn1(x)  # Batch normalization
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Using max pooling (2x2) to reduce spatial dimensions

        # Apply the second convolutional layer, activation, batch normalization, and max pooling
        x = self.conv2(x)  # Convolution operation
        x = self.relu2(x)  # Apply ReLU activation
        x = self.bn2(x)  # Batch normalization
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Using max pooling (2x2) to reduce spatial dimensions

        # Flatten the feature maps into a 1D feature vector for input to fully connected layers
        x = x.reshape(x.size(0), -1)  # Reshape to (batch_size, num_features)

        # Pass through the first fully connected layer and activation function
        x = self.fc1(x)  # Fully connected operation
        x = self.relu3(x)  # Apply ReLU activation

        # Pass through the second fully connected layer and activation function
        x = self.fc2(x)  # Fully connected operation
        x = self.relu4(x)  # Apply ReLU activation

        # Pass through the final fully connected layer to produce raw logits
        x = self.fc3(x)  # Fully connected operation (output layer)
        return x  # Return raw logits for use with CrossEntropyLoss

In [102]:
# model = LeNet(num_classes).to(device)
# input_size = (1, 32, 32)
# with torch.no_grad():
#     x = model.conv1(torch.randn(1, *input_size).to(device))
#     x = F.max_pool2d(x, kernel_size=2, stride=2)
#     x = model.conv2(x)
#     x = F.max_pool2d(x, kernel_size=2, stride=2)
#     print(x.shape)
#     print(16 * 5 *5)

# Implement K-Fold Cross Validation

In [103]:
CONFIG = {
    'k_folds': 5,  # Number of folds for K-Fold Cross Validation. This determines how the dataset will be split into training and validation subsets.
    'batch_size': 64,  # Batch size for training, which defines how many samples are processed before updating the model.
    'num_epochs': 10,  # Number of complete passes through the training dataset.
    'learning_rate': 0.001,  # Initial learning rate for the optimizer to control the step size in parameter updates.
}

DEFAULT_SUGGEST_PARAMS = {
    'fc1_range': (64, 256, 32),  # Range for the size of the first fully connected layer. Suggested as multiples of 32 between 64 and 256.
    'fc2_range': (32, 128, 16),  # Range for the size of the second fully connected layer. Suggested as multiples of 16 between 32 and 128.
    'batch_size_range': (32, 256, 32),  # Range for batch size suggestions. Suggested as multiples of 32 between 32 and 256.
    'learning_rate_range': (1e-5, 1e-1),  # Logarithmic range for suggesting learning rates between 1e-5 and 1e-1.
    'optimizers': ['Adam', 'SGD', 'Adadelta', 'Adagrad', 'RMSprop'],  # List of optimizers to choose from for training the model.
}

In [104]:
def initialize_training_pipeline(trial, num_classes, device, suggest_params=DEFAULT_SUGGEST_PARAMS):
    """
    Initializes the training pipeline by setting up the model, optimizer, criterion,
    and batch size based on parameters suggested by the Optuna trial object.

    Args:
    - trial (optuna.trial.Trial): The trial object that Optuna uses to suggest hyperparameter values.
    - num_classes (int): The number of classes for the classification task.
    - device (torch.device): The device (CPU or GPU) where the model and tensors will reside.
    - suggest_params (dict): A dictionary containing ranges and options for hyperparameter suggestions.

    Returns:
    - model (torch.nn.Module): The initialized model with suggested architecture based on trial.
    - optimizer (torch.optim.Optimizer): The optimizer selected and initialized with suggested learning rate.
    - criterion (torch.nn.Module): The loss function used for training the model.
    - batch_size (int): The batch size suggested for loading data during training.
    """

    # Suggest the size of the first fully connected layer (fc1) using a defined range.
    # Optuna uses this suggestion to explore different values to optimize the model's performance.
    # `trial.suggest_int` ensures that fc1_size is an integer.
    fc1_size = trial.suggest_int('fc1_size', low=suggest_params['fc1_range'][0], high=suggest_params['fc1_range'][1], step=suggest_params['fc1_range'][2])

    # Suggest the size of the second fully connected layer (fc2) using a defined range.
    # This allows for flexibility in adjusting the model's capacity.
    fc2_size = trial.suggest_int('fc2_size', low=suggest_params['fc2_range'][0], high=suggest_params['fc2_range'][1], step=suggest_params['fc2_range'][2])

    # Initialize the model with the suggested layer sizes and number of output classes (num_classes).
    # The model is moved to the specified device (GPU or CPU).
    model = LeNet(num_classes, fc1_size, fc2_size).to(device)

    # Suggest an optimizer from a predefined list.
    # Different optimizers may perform better depending on the training task and dataset.
    optimizer_name = trial.suggest_categorical('optimizer', suggest_params['optimizers'])

    # Suggest a learning rate in a logarithmic scale (log=True).
    # This helps explore a wide range of values for the learning rate.
    learning_rate = trial.suggest_float('lr', *suggest_params['learning_rate_range'], log=True)

    # Initialize the optimizer with the suggested learning rate and optimizer type.
    # The `getattr` function retrieves the optimizer from the PyTorch `optim` module.
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    # Suggest a batch size for data loading from a defined range.
    # Batch size affects the memory usage and stability of the training process.
    batch_size = trial.suggest_int('batch_size', low=suggest_params['batch_size_range'][0], high=suggest_params['batch_size_range'][1], step=suggest_params['batch_size_range'][2])

    # Define the loss function for classification.
    # CrossEntropyLoss is commonly used for multi-class classification problems.
    criterion = nn.CrossEntropyLoss()

    # Return the initialized model, optimizer, loss function, and batch size.
    return model, optimizer, criterion, batch_size

In [105]:
# This function calculates average metrics (loss and accuracy) based on the provided lists of values.
# It takes two input parameters:
# - losses: A list of numeric values representing loss metrics.
# - accuracies: A list of numeric values representing accuracy metrics.
# The function computes the arithmetic mean of the values in each list and returns the results as a tuple.
def calculate_metrics(losses, accuracies):
    avg_loss = sum(losses) / len(losses)
    avg_accuracy = sum(accuracies) / len(accuracies)
    return avg_loss, avg_accuracy

In [106]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Trains the model for one complete pass (epoch) over the provided training data.

    Args:
    - model (torch.nn.Module): The neural network model to train.
    - train_loader (torch.utils.data.DataLoader): The DataLoader that provides batches of training data.
    - criterion (torch.nn.Module): The loss function used to calculate the difference between predictions and targets.
    - optimizer (torch.optim.Optimizer): The optimizer responsible for updating the model's parameters.
    - device (torch.device): The device on which calculations will be performed (CPU or GPU).

    Returns:
    - avg_loss (float): The average loss across all training batches.
    - accuracy (float): The overall accuracy of the model on the training dataset.
    """
    model.train()  # Sets the model to training mode (enables features like dropout, batchnorm updates).
    total_loss, correct = 0.0, 0  # Initialize counters for total loss and total correct predictions.

    # Iterate through training data in batches
    for data, target in train_loader:
        # Move data and labels to the specified device (e.g., GPU for faster computations).
        data, target = data.to(device), target.to(device)

        # Perform a forward pass to get the model's predictions.
        output = model(data)

        # Compute the loss for the current batch using the criterion (e.g., CrossEntropyLoss).
        loss = criterion(output, target)

        # Zero the gradients of the optimizer to prevent accumulation from previous steps.
        optimizer.zero_grad()

        # Perform backpropagation to compute gradients for the model's parameters.
        loss.backward()

        # Update the model's parameters using the computed gradients.
        optimizer.step()

        # Accumulate the current batch's loss to the total_loss.
        total_loss += loss.item()

        # Determine the predicted class for each example in the batch.
        pred = output.argmax(dim=1)

        # Count the number of correct predictions by comparing with the true labels.
        correct += pred.eq(target).sum().item()

    # Calculate the average loss across all training batches.
    avg_loss = total_loss / len(train_loader)

    # Calculate the accuracy as the fraction of correct predictions over the total dataset.
    accuracy = correct / len(train_loader.dataset)

    # Return the average loss and accuracy for this epoch.
    return avg_loss, accuracy

In [107]:
def validate(model, val_loader, criterion, device):
    """
    Evaluates the model on the validation dataset to calculate the average loss and accuracy.

    Args:
    - model (torch.nn.Module): The trained neural network model to evaluate.
    - val_loader (torch.utils.data.DataLoader): DataLoader for the validation dataset.
    - criterion (torch.nn.Module): The loss function used to calculate the loss between predictions and targets.
    - device (torch.device): The device (e.g., CPU or GPU) where computations will take place.

    Returns:
    - avg_loss (float): The average loss over all validation batches.
    - accuracy (float): The overall accuracy of the model on the validation dataset.
    """

    # Set the model to evaluation mode, which disables certain behaviors like dropout to ensure consistent results.
    model.eval()

    # Initialize total variables to calculate cumulative loss and correct predictions.
    total_loss, correct = 0.0, 0

    # Disable gradient computation since the model is in evaluation mode to save memory and computation.
    with torch.no_grad():
        # Iterate over the validation dataset in batches
        for data, target in val_loader:
            # Move the input data and labels to the specified device (e.g., GPU for faster computation).
            data, target = data.to(device), target.to(device)

            # Perform a forward pass to get predictions from the model.
            output = model(data)

            # Compute the loss for the current batch.
            total_loss += criterion(output, target).item()

            # Get the predicted class for each input by finding the index of the maximum value along the last dimension.
            pred = output.argmax(dim=1, keepdim=True)

            # Increment the total count of correct predictions by comparing the predictions with the true labels.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calculate the average loss by dividing the total loss by the number of validation batches.
    avg_loss = total_loss / len(val_loader)

    # Calculate the accuracy by dividing the number of correct predictions by the total number of samples in the dataset.
    accuracy = correct / len(val_loader.dataset)

    # Return the average loss and the accuracy.
    return avg_loss, accuracy

In [108]:
# Initialize empty lists to track losses and accuracies for both training and validation
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

# Set up K-Fold Cross Validation with the number of splits from the CONFIG and shuffle enabled
kfold = KFold(n_splits=CONFIG['k_folds'], shuffle=True)


def objective(trial):
    """
    The objective function used in Optuna for hyperparameter optimization.

    Args:
    - trial: Optuna trial object that suggests hyperparameter values.

    Returns:
    - Average validation accuracy across all folds for the given trial's hyperparameter configuration.
    """
    try:
        # Initialize the training pipeline with the trial suggesting hyperparameters
        # This function should return the model, optimizer, criterion, and batch size
        model, optimizer, criterion, batch_size = initialize_training_pipeline(trial, num_classes, device)

        # Reinitialize K-Fold Cross Validation for the dataset splitting
        kfold = KFold(n_splits=CONFIG['k_folds'], shuffle=True)

        # Total validation accuracy to calculate the average later
        total_val_acc = 0.0

        # Loop through each fold in the k-fold cross-validation process
        for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
            print(f'Fold {fold + 1}')  # Print current fold number

            # Split dataset into training and validation subsets for the current fold
            train_subset, val_subset = Subset(dataset, train_idx), Subset(dataset, val_idx)

            # Create DataLoaders for training and validation subsets
            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)  # Shuffle training data
            val_loader = DataLoader(val_subset, batch_size=batch_size)  # No shuffle for validation data

            # Loop for each epoch within the current fold
            for epoch in range(CONFIG["num_epochs"]):
                print(f'Epoch {epoch + 1}')  # Print current epoch number

                # Perform one epoch of training and return the loss and accuracy
                train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)

                # Perform validation and return the loss and accuracy
                val_loss, val_acc = validate(model, val_loader, criterion, device)

                # Accumulate validation accuracy for calculating average
                total_val_acc += val_acc

                # Record losses and accuracies for analysis
                train_losses.append(train_loss)
                train_accuracies.append(train_acc)
                val_losses.append(val_loss)
                val_accuracies.append(val_acc)

                # Print the current training and validation performance
                print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
                print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Calculate average metrics for the entire training and validation process
        avg_train_loss, avg_train_acc = calculate_metrics(train_losses, train_accuracies)
        avg_val_loss, avg_val_acc = calculate_metrics(val_losses, val_accuracies)

        # Display the average metrics for this trial
        print(f'Average Train Loss: {avg_train_loss:.4f}, Average Train Acc: {avg_train_acc:.4f}')
        print(f'Average Val Loss: {avg_val_loss:.4f}, Average Val Acc: {avg_val_acc:.4f}')

        # Return the average validation accuracy normalized to the range [0,1]
        return total_val_acc / (CONFIG['k_folds'] * 100)

    except Exception as e:
        # Handle and log any exceptions that occur during the trial execution
        # Returning infinity ensures that this trial is considered poorly performing and not selected
        print(f"Trial failed due to: {str(e)}")
        # Check the optimization direction of the study
        if study.direction == 'maximize':
            return -float('inf')  # Very poor outcome for maximization
        else:
            return float('inf')  # Very poor outcome for minimization

In [109]:
# Create an Optuna study to perform hyperparameter optimization. The objective is set to minimize a metric.
study = optuna.create_study(direction='maximize')

# Set the logging level for Optuna to DEBUG to output detailed information during the optimization process.
optuna.logging.set_verbosity(optuna.logging.DEBUG)

# Run the optimization process using the defined objective function with 20 trials.
study.optimize(objective, n_trials=20)

# Check if there are any completed trials and whether the study has a valid best trial.
if len(study.trials) > 0 and study.best_trial is not None:
    # Retrieve the best trial from the study and print its objective value and hyperparameters.
    trial = study.best_trial
    print('Best objective value: {}'.format(trial.value))
    print('Best hyperparameters: {}'.format(trial.params))
else:
    # Notify the user that no trials have completed successfully. The objective function might need fixing.
    print("No trials completed successfully. Please check your objective function.")

[I 2025-02-16 22:20:23,792] A new study created in memory with name: no-name-5f7412a1-4258-4982-bdeb-3fda3e113523


Fold 1
Epoch 1
Train Loss: 1.7614, Train Acc: 0.5016
Val Loss: 1.0796, Val Acc: 0.7172
Epoch 2
Train Loss: 0.7915, Train Acc: 0.7713
Val Loss: 0.6304, Val Acc: 0.8061
Epoch 3
Train Loss: 0.5562, Train Acc: 0.8297
Val Loss: 0.5007, Val Acc: 0.8426
Epoch 4
Train Loss: 0.4540, Train Acc: 0.8597
Val Loss: 0.4192, Val Acc: 0.8683
Epoch 5
Train Loss: 0.3891, Train Acc: 0.8805
Val Loss: 0.3709, Val Acc: 0.8861
Epoch 6
Train Loss: 0.3492, Train Acc: 0.8914
Val Loss: 0.3290, Val Acc: 0.8993
Epoch 7
Train Loss: 0.3122, Train Acc: 0.9031
Val Loss: 0.3035, Val Acc: 0.9072
Epoch 8
Train Loss: 0.2886, Train Acc: 0.9119
Val Loss: 0.2793, Val Acc: 0.9142
Epoch 9
Train Loss: 0.2674, Train Acc: 0.9173
Val Loss: 0.2590, Val Acc: 0.9175
Epoch 10
Train Loss: 0.2520, Train Acc: 0.9230
Val Loss: 0.2499, Val Acc: 0.9237
Fold 2
Epoch 1
Train Loss: 0.2374, Train Acc: 0.9275
Val Loss: 0.2430, Val Acc: 0.9259
Epoch 2
Train Loss: 0.2267, Train Acc: 0.9295
Val Loss: 0.2357, Val Acc: 0.9265
Epoch 3
Train Loss: 0.213

[I 2025-02-16 22:28:57,781] Trial 0 finished with value: 0.09362333333333331 and parameters: {'fc1_size': 160, 'fc2_size': 96, 'optimizer': 'Adam', 'lr': 5.071482846224742e-05, 'batch_size': 192}. Best is trial 0 with value: 0.09362333333333331.


Train Loss: 0.1071, Train Acc: 0.9663
Val Loss: 0.1141, Val Acc: 0.9651
Average Train Loss: 0.2290, Average Train Acc: 0.9301
Average Val Loss: 0.2083, Average Val Acc: 0.9362
Fold 1
Epoch 1
Train Loss: 2.3100, Train Acc: 0.0891
Val Loss: 2.3034, Val Acc: 0.0945
Epoch 2
Train Loss: 2.2981, Train Acc: 0.1010
Val Loss: 2.2924, Val Acc: 0.1117
Epoch 3
Train Loss: 2.2866, Train Acc: 0.1210
Val Loss: 2.2804, Val Acc: 0.1407
Epoch 4
Train Loss: 2.2756, Train Acc: 0.1517
Val Loss: 2.2701, Val Acc: 0.1718
Epoch 5
Train Loss: 2.2645, Train Acc: 0.1851
Val Loss: 2.2573, Val Acc: 0.2012
Epoch 6
Train Loss: 2.2525, Train Acc: 0.2154
Val Loss: 2.2455, Val Acc: 0.2293
Epoch 7
Train Loss: 2.2403, Train Acc: 0.2438
Val Loss: 2.2352, Val Acc: 0.2550
Epoch 8
Train Loss: 2.2281, Train Acc: 0.2693
Val Loss: 2.2220, Val Acc: 0.2748
Epoch 9
Train Loss: 2.2147, Train Acc: 0.2925
Val Loss: 2.2091, Val Acc: 0.3035
Epoch 10
Train Loss: 2.2005, Train Acc: 0.3190
Val Loss: 2.1951, Val Acc: 0.3242
Fold 2
Epoch 1
T

[I 2025-02-16 22:37:36,457] Trial 1 finished with value: 0.04970433333333332 and parameters: {'fc1_size': 224, 'fc2_size': 112, 'optimizer': 'Adadelta', 'lr': 0.0003823923546226007, 'batch_size': 192}. Best is trial 0 with value: 0.09362333333333331.


...

Train Loss: 0.0359, Train Acc: 0.9880
Val Loss: 0.0379, Val Acc: 0.9875
Average Train Loss: 0.6844, Average Train Acc: 0.7660
Average Val Loss: 0.6788, Average Val Acc: 0.7679
Fold 1
Epoch 1
Train Loss: 0.2655, Train Acc: 0.9160
Val Loss: 0.1479, Val Acc: 0.9520
Epoch 2
Train Loss: 0.1260, Train Acc: 0.9603
Val Loss: 0.1161, Val Acc: 0.9632
Epoch 3
Train Loss: 0.1042, Train Acc: 0.9670
Val Loss: 0.1016, Val Acc: 0.9673
Epoch 4
Train Loss: 0.0900, Train Acc: 0.9719
Val Loss: 0.0942, Val Acc: 0.9698
Epoch 5
Train Loss: 0.0832, Train Acc: 0.9737
Val Loss: 0.0845, Val Acc: 0.9736
Epoch 6
Train Loss: 0.0765, Train Acc: 0.9762
Val Loss: 0.0842, Val Acc: 0.9748
Epoch 7
Train Loss: 0.0727, Train Acc: 0.9765
Val Loss: 0.0741, Val Acc: 0.9770
Epoch 8
Train Loss: 0.0699, Train Acc: 0.9769
Val Loss: 0.0770, Val Acc: 0.9750
Epoch 9
Train Loss: 0.0638, Train Acc: 0.9784
Val Loss: 0.0773, Val Acc: 0.9762
Epoch 10
Train Loss: 0.0647, Train Acc: 0.9795
Val Loss: 0.0730, Val Acc: 0.9768
Fold 2
Epoch 1
T

[I 2025-02-17 01:07:19,255] Trial 18 finished with value: 0.09828366666666663 and parameters: {'fc1_size': 256, 'fc2_size': 48, 'optimizer': 'Adagrad', 'lr': 0.025374308187755247, 'batch_size': 32}. Best is trial 18 with value: 0.09828366666666663.


Train Loss: 0.0338, Train Acc: 0.9888
Val Loss: 0.0344, Val Acc: 0.9888
Average Train Loss: 0.6513, Average Train Acc: 0.7774
Average Val Loss: 0.6459, Average Val Acc: 0.7792
Fold 1
Epoch 1
Train Loss: 0.2537, Train Acc: 0.9185
Val Loss: 0.1597, Val Acc: 0.9501
Epoch 2
Train Loss: 0.1281, Train Acc: 0.9590
Val Loss: 0.1193, Val Acc: 0.9619
Epoch 3
Train Loss: 0.1034, Train Acc: 0.9672
Val Loss: 0.1079, Val Acc: 0.9648
Epoch 4
Train Loss: 0.0906, Train Acc: 0.9716
Val Loss: 0.0972, Val Acc: 0.9701
Epoch 5
Train Loss: 0.0844, Train Acc: 0.9731
Val Loss: 0.0904, Val Acc: 0.9716
Epoch 6
Train Loss: 0.0770, Train Acc: 0.9752
Val Loss: 0.0926, Val Acc: 0.9701
Epoch 7
Train Loss: 0.0727, Train Acc: 0.9771
Val Loss: 0.0795, Val Acc: 0.9748
Epoch 8
Train Loss: 0.0689, Train Acc: 0.9780
Val Loss: 0.0815, Val Acc: 0.9748
Epoch 9
Train Loss: 0.0688, Train Acc: 0.9778
Val Loss: 0.0812, Val Acc: 0.9749
Epoch 10
Train Loss: 0.0648, Train Acc: 0.9792
Val Loss: 0.0790, Val Acc: 0.9756
Fold 2
Epoch 1
T

[I 2025-02-17 01:16:57,329] Trial 19 finished with value: 0.09819733333333337 and parameters: {'fc1_size': 224, 'fc2_size': 48, 'optimizer': 'Adagrad', 'lr': 0.01707678659667944, 'batch_size': 32}. Best is trial 18 with value: 0.09828366666666663.


Train Loss: 0.0369, Train Acc: 0.9880
Val Loss: 0.0377, Val Acc: 0.9870
Average Train Loss: 0.6216, Average Train Acc: 0.7876
Average Val Loss: 0.6164, Average Val Acc: 0.7893
Best objective value: 0.09828366666666663
Best hyperparameters: {'fc1_size': 256, 'fc2_size': 48, 'optimizer': 'Adagrad', 'lr': 0.025374308187755247, 'batch_size': 32}
